In [1]:
import gym
from gym import error, spaces, utils
from gym.utils import seeding

In [2]:
import os
import pybullet as p
import pybullet_data

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import math
import random

In [5]:
class BikeEnv(gym.Env):
    def __init__(self):
        p.connect(p.GUI)
        p.resetDebugVisualizerCamera(cameraDistance=1.5, cameraYaw=0, cameraPitch=-40, cameraTargetPosition=[0.55,-0.35,0.2])
        self.action_space = spaces.Box(np.array([-1,0,-1,0,-1,0]),np.array([1,1,1,1,1,1]))#[a,a,b,b,c,c]
        self.observation_space = spaces.Box(np.array([-1000, -1000 , 0]), np.array([1000 , 1000 , 10]))
        
    def step(self, action):
        #if action[0] >= 5 : action[0]=0 
            
        p.configureDebugVisualizer(p.COV_ENABLE_SINGLE_STEP_RENDERING)
        #p.setJointMotorControl2(self.pid, 0, p.VELOCITY_CONTROL, targetVelocity=action[0], force=action[1])
        p.setJointMotorControl2(self.pid, 1, p.VELOCITY_CONTROL, targetVelocity=action[2], force=action[3])
        p.setJointMotorControl2(self.pid, 2, p.VELOCITY_CONTROL, targetVelocity=action[4], force=action[5])
        p.stepSimulation()
        
        state = p.getLinkState(self.pid,0)[0]
        if state[2] <= 0.5 :
            reward = -100
            done = True
        else :
            reward = math.sqrt((self.origin[0]-state[0])**2+(self.origin[1]-state[1])**2)
            done = False
        #self.origin = state    
        observation = p.getLinkState(self.pid,0)[0] #+ p.getJointState(self.pid,0)[:2] + p.getJointState(self.pid,1)[:2] + p.getJointState(self.pid,2)[:2]
        state_object, _ = p.getBasePositionAndOrientation(self.pid)
        info = {'x':state_object[0],'y':state_object[1],'z':state_object[2]}
        print(p.getBasePositionAndOrientation(self.pid))
        return observation, reward, done, info
            
        
    def reset(self):
        p.resetSimulation()
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)
        p.setGravity(0,0,-10)
        urdfRootPath = pybullet_data.getDataPath()
        planeUid = p.loadURDF(os.path.join(urdfRootPath,"plane.urdf"), basePosition=[0,0,0])
        self.pid = p.loadURDF(os.path.join(urdfRootPath, "bicycle/bike.urdf"),basePosition=[0,0,1])
        self.origin = p.getLinkState(self.pid,0)[0]
        
        observation = p.getLinkState(self.pid,0)[0] #+ p.getJointState(self.pid,0)[:2] + p.getJointState(self.pid,1)[:2] + p.getJointState(self.pid,2)[:2]
        
        p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
        return observation
        
    def render(self, mode='human'):
        view_matrix = p.computeViewMatrixFromYawPitchRoll(cameraTargetPosition=[0.7,0,0.05],
                                                            distance=.7,
                                                            yaw=90,
                                                            pitch=-70,
                                                            roll=0,
                                                            upAxisIndex=2)
        proj_matrix = p.computeProjectionMatrixFOV(fov=60,
                                                     aspect=float(960) /720,
                                                     nearVal=0.1,
                                                     farVal=100.0)
        (_, _, px, _, _) = p.getCameraImage(width=960,
                                              height=720,
                                              viewMatrix=view_matrix,
                                              projectionMatrix=proj_matrix,
                                              renderer=p.ER_BULLET_HARDWARE_OPENGL)

        rgb_array = np.array(px, dtype=np.uint8)
        rgb_array = np.reshape(rgb_array, (720,960, 4))

        rgb_array = rgb_array[:, :, :3]
        return rgb_array

    def close(self):
        p.disconnect()

In [6]:
env = BikeEnv()
for i_episode in range(20):
    observation = env.reset()
    for t in range(1000):
        env.render()
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print(info)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

C:\Users\SERVER\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


((1.7959433193512906e-07, 6.304808000459502e-11, 0.9998265363350956), (0.7068251810577677, 6.450004238519977e-07, -6.449232045547803e-07, 0.7073882692141722))
{'x': 1.7959433193512906e-07, 'y': 6.304808000459502e-11, 'z': 0.9998265363350956}
((-2.748172834217612e-07, 7.304048512966058e-11, 0.9994789711850448), (0.7068251805964075, -9.870190311355405e-07, 9.868318155470083e-07, 0.7073882696743761))
{'x': -2.748172834217612e-07, 'y': 7.304048512966058e-11, 'z': 0.9994789711850448}
((-1.1788435715115611e-06, -5.231483592801366e-11, 0.998957488473713), (0.7068251801648189, -4.233812430570949e-06, 4.2331583318690364e-06, 0.7073882700816622))
{'x': -1.1788435715115611e-06, 'y': -5.231483592801366e-11, 'z': 0.998957488473713}
((-2.4346116223001838e-06, -2.1351219183054512e-10, 0.9982622034894734), (0.706825179496249, -8.743916345673856e-06, 8.742608460166017e-06, 0.7073882706669699))
{'x': -2.4346116223001838e-06, 'y': -2.1351219183054512e-10, 'z': 0.9982622034894734}
((-3.5801268063999002e-0

((-0.0008244922515008228, 0.0030940363722343914, 0.9883044607419847), (0.7056731325835415, -0.0020102400009199208, 0.002098875460374032, 0.7085315685321805))
{'x': -0.0008244922515008228, 'y': 0.0030940363722343914, 'z': 0.9883044607419847}
((-0.0008432823220945828, 0.0032882534476459356, 0.9882987883024824), (0.7056035284573743, -0.0020096185315899837, 0.002098608811830828, 0.7086008875967598))
{'x': -0.0008432823220945828, 'y': 0.0032882534476459356, 'z': 0.9882987883024824}
((-0.000863836330652262, 0.0034880866764088803, 0.9882929475194144), (0.7055319043981009, -0.002008948542942364, 0.0020983032032479815, 0.7086722042847513))
{'x': -0.000863836330652262, 'y': 0.0034880866764088803, 'z': 0.9882929475194144}
((-0.0008825135651543449, 0.003693564243435505, 0.9882868295388606), (0.7054582486000929, -0.0020083160918036415, 0.0020980471504851834, 0.7087455286255563))
{'x': -0.0008825135651543449, 'y': 0.003693564243435505, 'z': 0.9882868295388606}
((-0.0008937568513900238, 0.00390471166

((-0.0006540162019748555, 0.013899272990485986, 0.9878958293263751), (0.7017889377891074, -0.0020031864703314787, 0.0021135609852572603, 0.7123789770204924))
{'x': -0.0006540162019748555, 'y': 0.013899272990485986, 'z': 0.9878958293263751}
((-0.0006438464779244089, 0.014321959002639642, 0.98787822377208), (0.7016365242590471, -0.0020026664487293707, 0.0021139252198402866, 0.712529093071254))
{'x': -0.0006438464779244089, 'y': 0.014321959002639642, 'z': 0.98787822377208}
((-0.0006392158518602969, 0.014751970944050133, 0.987859746300539), (0.701481431741615, -0.002002182628657993, 0.002114341066165674, 0.7126817815465126))
{'x': -0.0006392158518602969, 'y': 0.014751970944050133, 'z': 0.987859746300539}
((-0.0006430539757488121, 0.015189372428136491, 0.9878402814296912), (0.7013236355127095, -0.002001766208102294, 0.002114835778676426, 0.7128370632009208))
{'x': -0.0006430539757488121, 'y': 0.015189372428136491, 'z': 0.9878402814296912}
((-0.0006500503297716563, 0.01563424193475561, 0.987

((-0.0006610332319313567, 0.03578173176176374, 0.9867133412938702), (0.6938522165992024, -0.0019809120482480896, 0.002136381439887401, 0.7201115284330435))
{'x': -0.0006610332319313567, 'y': 0.03578173176176374, 'z': 0.9867133412938702}
((-0.0006630827517749142, 0.03654136488499272, 0.986663465205083), (0.6935749754870483, -0.001980143621191969, 0.002137192214753369, 0.7203785566067448))
{'x': -0.0006630827517749142, 'y': 0.03654136488499272, 'z': 0.986663465205083}
((-0.0006640027842872319, 0.03731181450857838, 0.9866123410348432), (0.693293668227838, -0.0019793533441218687, 0.0021380037838515395, 0.7206492910531085))
{'x': -0.0006640027842872319, 'y': 0.03731181450857838, 'z': 0.9866123410348432}
((-0.0006652928776585018, 0.038093203003933415, 0.9865598597748738), (0.6930082543494226, -0.0019785679064962783, 0.0021388145539817088, 0.7209237616731115))
{'x': -0.0006652928776585018, 'y': 0.038093203003933415, 'z': 0.9865598597748738}
((-0.0006645030141117034, 0.03888564486848872, 0.986

...

((-0.0006221099896060834, 0.1522305041560979, 0.9724178260474383), (0.6499083044982479, -0.0018546057865195028, 0.0022398944179856596, 0.760007065134651))
{'x': -0.0006221099896060834, 'y': 0.1522305041560979, 'z': 0.9724178260474383}
((-0.0005618867492246098, 0.15448607966663377, 0.9720240663421215), (0.6490266589861049, -0.0018516683791684463, 0.0022415749406933874, 0.7607601084384835))
{'x': -0.0005618867492246098, 'y': 0.15448607966663377, 'z': 0.9720240663421215}
((-0.0005143352507982034, 0.15676781550230934, 0.9716194937275644), (0.6481334948352211, -0.0018487947069173293, 0.002243388219508935, 0.7615211894885859))
{'x': -0.0005143352507982034, 'y': 0.15676781550230934, 'z': 0.9716194937275644}
((-0.00047693476654946663, 0.15907604162568117, 0.9712039968900429), (0.6472286402288985, -0.0018459642362847397, 0.002245313993719286, 0.7622903897128437))
{'x': -0.00047693476654946663, 'y': 0.15907604162568117, 'z': 0.9712039968900429}
((-0.00044662583444582697, 0.16141109659047226, 0.9

((-0.0029760042008119035, 0.25911288665066745, 0.9475138374767292), (0.6066731151342784, -0.001714068343951281, 0.002318887113580402, 0.794946171828972))
{'x': -0.0029760042008119035, 'y': 0.25911288665066745, 'z': 0.9475138374767292}
((-0.0030822341364957753, 0.2625971866024079, 0.9464923397530639), (0.6052107889381958, -0.0017077156251974243, 0.0023191898941497214, 0.7960600517664386))
{'x': -0.0030822341364957753, 'y': 0.2625971866024079, 'z': 0.9464923397530639}
((-0.0031820696016975334, 0.266120362521866, 0.9454450723932323), (0.6037284461699427, -0.0017014542764922586, 0.002319697550177772, 0.7971848514252183))
{'x': -0.0031820696016975334, 'y': 0.266120362521866, 'z': 0.9454450723932323}
((-0.003278867523355089, 0.26968277824805814, 0.9443715159284735), (0.602225793321588, -0.0016952094010171416, 0.002320313319459052, 0.7983206350016078))
{'x': -0.003278867523355089, 'y': 0.26968277824805814, 'z': 0.9443715159284735}
((-0.0033743365564299576, 0.27328477850896904, 0.9432710751528

((-0.004884631298900585, 0.4215172716449165, 0.884730398888485), (0.5341783463150398, -0.001478277438519305, 0.002412474197504674, 0.8453670735202492))
{'x': -0.004884631298900585, 'y': 0.4215172716449165, 'z': 0.884730398888485}
((-0.004898714756620928, 0.42668258865202635, 0.882214164454231), (0.5317083844199297, -0.0014712475833934017, 0.0024165273173240914, 0.8469227767416704))
{'x': -0.004898714756620928, 'y': 0.42668258865202635, 'z': 0.882214164454231}
((-0.004907890469673351, 0.4318942882400478, 0.8796373829563545), (0.5292045027982161, -0.0014642047039174033, 0.0024207429509161093, 0.8484895935285497))
{'x': -0.004907890469673351, 'y': 0.4318942882400478, 'z': 0.8796373829563545}
((-0.0049202363664579, 0.43715246805271935, 0.8769989075340915), (0.5266662024442664, -0.0014570087508499591, 0.002424898605193104, 0.850067472730953))
{'x': -0.0049202363664579, 'y': 0.43715246805271935, 'z': 0.8769989075340915}
((-0.004930280982379472, 0.44245703045641205, 0.8742971983688479), (0.52

((-0.004705271741377679, 0.6419162965841138, 0.7395607218469369), (0.4159053778939905, -0.0011496919209067055, 0.002600301113031704, 0.9094034491255603))
{'x': -0.004705271741377679, 'y': 0.6419162965841138, 'z': 0.7395607218469369}
((-0.004665215362104821, 0.6485939857147887, 0.7337681866966403), (0.41179508598356035, -0.0011390019087203218, 0.002606930609221766, 0.9112720305963762))
{'x': -0.004665215362104821, 'y': 0.6485939857147887, 'z': 0.7337681866966403}
((-0.004627664769488793, 0.6552945930382349, 0.727849172866396), (0.4076295943860931, -0.0011281176012304056, 0.0026135084014246544, 0.913142930052652))
{'x': -0.004627664769488793, 'y': 0.6552945930382349, 'z': 0.727849172866396}
((-0.004589222219327413, 0.6620164053946652, 0.7218013393383367), (0.4034082736592887, -0.001117079817533299, 0.0026201289935129674, 0.9150156565873455))
{'x': -0.004589222219327413, 'y': 0.6620164053946652, 'z': 0.7218013393383367}
((-0.004552016523516932, 0.6687578565317451, 0.7156226160639289), (0.

KeyboardInterrupt: 